In [1]:
from openai import OpenAI


openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [2]:
"""

问题分解：分解成可以执行的顺序子问题


采用的命令：vllm serve  /home/featurize/work/llm/models/Qwen2.5-32B-Instruct-AWQ --served-model-name Qwen2.5-7B-Instruct --quantization awq   --disable-frontend-multiprocessing --max-model-len 2048 

使用模型：Qwen2.5-32B-Instruct-AWQ 

下面的Prompt是在qwen2.5-turbo-1M 中测试通过的Prompt提示词。



"""
print()

In [3]:
def create_query_plan(query):

    system_prompt = "You are a helpful assistant."


    user_prompt = """你的任务是将一个复杂问题分解为多个独立的子问题，并以结构化的方式提供子问题及其依赖关系。输出内容应严格按照以下 JSON 格式，便于系统解析和后续处理：

    输入问题：<用户复杂问题>
    输出：
    {
      "main_question": "<原始复杂问题>",
      "sub_questions": [
        {
          "id": 1,
          "text": "<子问题1>",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "<子问题2>",
          "dependencies": [1]
        },
        {
          "id": 3,
          "text": "<子问题3>",
          "dependencies": [2]
        }
      ]
    }
    要求：
    1. 子问题应独立且可回答，避免歧义。
    2. 如果子问题的答案需要依赖其他子问题，请在 "dependencies" 字段中列出其依赖的子问题的 ID。
    3. 确保所有子问题的顺序和逻辑清晰。
    
    
    示例：
    输入问题：如何设计一个供大型企业使用的文档搜索引擎？
    输出：
    {
      "main_question": "如何设计一个供大型企业使用的文档搜索引擎？",
      "sub_questions": [
        {
          "id": 1,
          "text": "目标用户有哪些核心需求？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "企业需要支持哪些文档类型（如PDF、Word、图像等）？",
          "dependencies": []
        },
        {
          "id": 3,
          "text": "文档搜索需要实现哪些核心功能（如全文检索、关键词匹配、语义搜索）？",
          "dependencies": [1, 2]
        },
        {
          "id": 4,
          "text": "系统如何确保安全性和权限管理？",
          "dependencies": []
        },
        {
          "id": 5,
          "text": "数据的更新频率和处理规模如何规划？",
          "dependencies": []
        },
        {
          "id": 6,
          "text": "搜索结果的展示形式如何优化用户体验？",
          "dependencies": [3]
        }
      ]
    }
    
    
    输入问题：
    """

    user_prompt_kbqa = """你的任务是将用户的复杂问题分解为多个独立的子问题，以便从知识库中检索答案。子问题应具有明确的检索目标（如实体、关系或属性），并以 JSON 格式输出，确保结构化、易解析。

    输出格式：
    {
      "main_question": "<用户复杂问题>",
      "sub_questions": [
        {
          "id": 1,
          "text": "<子问题1>",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "<子问题2>",
          "dependencies": [1]
        },
        ...
      ]
    }
    
    以下是一些知识库问答场景的示例：
    
    **示例1: 简单事实型问答**
    输入问题：乔布斯创立的公司有哪些？
    输出：
    {
      "main_question": "乔布斯创立的公司有哪些？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯是谁？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "乔布斯创立了哪些公司？",
          "dependencies": [1]
        }
      ]
    }
    
    **示例2: 复杂组合型问答**
    输入问题：乔布斯创立的公司中，目前估值最高的是哪家？
    输出：
    {
      "main_question": "乔布斯创立的公司中，目前估值最高的是哪家？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯创立了哪些公司？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "这些公司各自的估值是多少？",
          "dependencies": [1]
        },
        {
          "id": 3,
          "text": "估值最高的公司是哪家？",
          "dependencies": [2]
        }
      ]
    }
    
    **示例3: 因果推理型问答**
    输入问题：为什么乔布斯离开苹果公司？
    输出：
    {
      "main_question": "为什么乔布斯离开苹果公司？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯在苹果公司的职位是什么？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "乔布斯离开苹果公司的时间是什么时候？",
          "dependencies": []
        },
        {
          "id": 3,
          "text": "当时苹果公司内部发生了哪些影响乔布斯离职的事件？",
          "dependencies": [1, 2]
        },
        {
          "id": 4,
          "text": "这些事件如何导致乔布斯的离职？",
          "dependencies": [3]
        }
      ]
    }
    
    **示例4: 时间相关的知识检索型问答**
    输入问题：特斯拉的创始人创立了哪些其他公司？
    输出：
    {
      "main_question": "特斯拉的创始人创立了哪些其他公司？",
      "sub_questions": [
        {
          "id": 1,
          "text": "特斯拉的创始人是谁？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "这些创始人创立了哪些公司？",
          "dependencies": [1]
        }
      ]
    }
    
    **示例5: 多跳推理型问答**
    输入问题：乔布斯的合伙人现在在哪家公司任职？
    输出：
    {
      "main_question": "乔布斯的合伙人现在在哪家公司任职？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯有哪些合伙人？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "这些合伙人分别在哪家公司任职？",
          "dependencies": [1]
        }
      ]
    }
    
    **注意事项：**
    1. 子问题应简单、明确，便于直接检索答案。
    2. 如果子问题依赖于其他子问题的答案，请通过 `dependencies` 字段进行明确标注。
    3. 子问题的顺序应与其逻辑依赖关系保持一致。
    4. 子问题应避免重复，但要确保问题分解完整。
    

    
    用户问题：
    """

    # system_prompt = """您是一个为问题创建有序解决步骤的AI专家助手，给定一个问题，可以为该问题创建一系列子问题，并且子问题的答案经过推理可以回答原始问题。
    # **思考推理**
    # - 你需要思考原始问题的解决步骤，并能够产生最终的答案
    # - 解决步骤中需要包含原始问题的子问题，每个子问题之间具有连续性和关联性
    # - 思考和推理解决问题的步骤要模拟程序员的思维
    # - 解决问题的步骤可以参考数据库表设计的逻辑
    
    # 每个子问题的要求：
    # - 子问题列表包含若干个独立的子问题，每个子问题需要可以单独被回答
    # - 子问题列表的答案可以推理出回答原始问题的答案
    # - 仅回答子问题列表，其它内容不要回答
    # - 不要重复子问题
    # - 子问题的粒度要适用，不要过于细致
    # - 不要产生与原始问题无关的子问题

    # # **样例**
    # # 原始问题：姚明和张三的身高谁高
    # # 子问题列表：['姚明身高是多少', '张三身高是多少', '姚明和张三的身高谁高']
    
    # """

    # """
    # **重点**
    # - 检查子问题列表的答案是否可以回答原始问题。
    # - 自己要有自我纠错的机制
    # - 注意子问题推理的逻辑
    # """

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt + query},
    ]
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_kbqa + query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       # "guide_json": Query.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content



In [4]:
def create_query_plan(query):

    system_prompt = "You are a helpful assistant."


    # user_prompt = """你的任务是将一个复杂问题分解为多个独立的子问题，并以结构化的方式提供子问题及其依赖关系。输出内容应严格按照以下 JSON 格式，便于系统解析和后续处理：

    # 输入问题：<用户复杂问题>
    # 输出：
    # {
    #   "main_question": "<原始复杂问题>",
    #   "sub_questions": [
    #     {
    #       "id": 1,
    #       "text": "<子问题1>",
    #       "dependencies": []
    #     },
    #     {
    #       "id": 2,
    #       "text": "<子问题2>",
    #       "dependencies": [1]
    #     },
    #     {
    #       "id": 3,
    #       "text": "<子问题3>",
    #       "dependencies": [2]
    #     }
    #   ]
    # }
    # 要求：
    # 1. 子问题应独立且可回答，避免歧义。
    # 2. 如果子问题的答案需要依赖其他子问题，请在 "dependencies" 字段中列出其依赖的子问题的 ID。
    # 3. 确保所有子问题的顺序和逻辑清晰。
    
    
    # 示例：
    # 输入问题：如何设计一个供大型企业使用的文档搜索引擎？
    # 输出：
    # {
    #   "main_question": "如何设计一个供大型企业使用的文档搜索引擎？",
    #   "sub_questions": [
    #     {
    #       "id": 1,
    #       "text": "目标用户有哪些核心需求？",
    #       "dependencies": []
    #     },
    #     {
    #       "id": 2,
    #       "text": "企业需要支持哪些文档类型（如PDF、Word、图像等）？",
    #       "dependencies": []
    #     },
    #     {
    #       "id": 3,
    #       "text": "文档搜索需要实现哪些核心功能（如全文检索、关键词匹配、语义搜索）？",
    #       "dependencies": [1, 2]
    #     },
    #     {
    #       "id": 4,
    #       "text": "系统如何确保安全性和权限管理？",
    #       "dependencies": []
    #     },
    #     {
    #       "id": 5,
    #       "text": "数据的更新频率和处理规模如何规划？",
    #       "dependencies": []
    #     },
    #     {
    #       "id": 6,
    #       "text": "搜索结果的展示形式如何优化用户体验？",
    #       "dependencies": [3]
    #     }
    #   ]
    # }
    
    
    # 输入问题：
    # """

    system_prompt = """你的任务是将用户的复杂问题分解为多个独立的子问题，以便从知识库中检索答案。子问题应具有明确的检索目标（如实体、关系或属性），并以 JSON 格式输出，确保结构化、易解析。

    输出格式：
    {
      "main_question": "<用户复杂问题>",
      "sub_questions": [
        {
          "id": 1,
          "text": "<子问题1>",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "<子问题2>",
          "dependencies": [1]
        },
        ...
      ]
    }
    
    以下是一些知识库问答场景的示例：
    
    **示例1: 简单事实型问答**
    输入问题：乔布斯创立的公司有哪些？
    输出：
    {
      "main_question": "乔布斯创立的公司有哪些？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯是谁？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "乔布斯创立了哪些公司？",
          "dependencies": [1]
        }
      ]
    }
    
    **示例2: 复杂组合型问答**
    输入问题：乔布斯创立的公司中，目前估值最高的是哪家？
    输出：
    {
      "main_question": "乔布斯创立的公司中，目前估值最高的是哪家？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯创立了哪些公司？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "这些公司各自的估值是多少？",
          "dependencies": [1]
        },
        {
          "id": 3,
          "text": "估值最高的公司是哪家？",
          "dependencies": [2]
        }
      ]
    }
    
    **示例3: 因果推理型问答**
    输入问题：为什么乔布斯离开苹果公司？
    输出：
    {
      "main_question": "为什么乔布斯离开苹果公司？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯在苹果公司的职位是什么？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "乔布斯离开苹果公司的时间是什么时候？",
          "dependencies": []
        },
        {
          "id": 3,
          "text": "当时苹果公司内部发生了哪些影响乔布斯离职的事件？",
          "dependencies": [1, 2]
        },
        {
          "id": 4,
          "text": "这些事件如何导致乔布斯的离职？",
          "dependencies": [3]
        }
      ]
    }
    
    **示例4: 时间相关的知识检索型问答**
    输入问题：特斯拉的创始人创立了哪些其他公司？
    输出：
    {
      "main_question": "特斯拉的创始人创立了哪些其他公司？",
      "sub_questions": [
        {
          "id": 1,
          "text": "特斯拉的创始人是谁？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "这些创始人创立了哪些公司？",
          "dependencies": [1]
        }
      ]
    }
    
    **示例5: 多跳推理型问答**
    输入问题：乔布斯的合伙人现在在哪家公司任职？
    输出：
    {
      "main_question": "乔布斯的合伙人现在在哪家公司任职？",
      "sub_questions": [
        {
          "id": 1,
          "text": "乔布斯有哪些合伙人？",
          "dependencies": []
        },
        {
          "id": 2,
          "text": "这些合伙人分别在哪家公司任职？",
          "dependencies": [1]
        }
      ]
    }
    
    **注意事项：**
    1. 子问题应简单、明确，便于直接检索答案。
    2. 如果子问题依赖于其他子问题的答案，请通过 `dependencies` 字段进行明确标注。
    3. 子问题的顺序应与其逻辑依赖关系保持一致。
    4. 子问题应避免重复，但要确保问题分解完整。
    
    """

    # system_prompt = """您是一个为问题创建有序解决步骤的AI专家助手，给定一个问题，可以为该问题创建一系列子问题，并且子问题的答案经过推理可以回答原始问题。
    # **思考推理**
    # - 你需要思考原始问题的解决步骤，并能够产生最终的答案
    # - 解决步骤中需要包含原始问题的子问题，每个子问题之间具有连续性和关联性
    # - 思考和推理解决问题的步骤要模拟程序员的思维
    # - 解决问题的步骤可以参考数据库表设计的逻辑
    
    # 每个子问题的要求：
    # - 子问题列表包含若干个独立的子问题，每个子问题需要可以单独被回答
    # - 子问题列表的答案可以推理出回答原始问题的答案
    # - 仅回答子问题列表，其它内容不要回答
    # - 不要重复子问题
    # - 子问题的粒度要适用，不要过于细致
    # - 不要产生与原始问题无关的子问题

    # # **样例**
    # # 原始问题：姚明和张三的身高谁高
    # # 子问题列表：['姚明身高是多少', '张三身高是多少', '姚明和张三的身高谁高']
    
    # """

    # """
    # **重点**
    # - 检查子问题列表的答案是否可以回答原始问题。
    # - 自己要有自我纠错的机制
    # - 注意子问题推理的逻辑
    # """

    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":  query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       # "guide_json": Query.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content



In [5]:
"""
上述的Prompt能够满足需求，但是由于给定的Prompt太长，导致解码的时间太长，需要缩减Prompt的长度。

目的：为达到上述缩减Prompt长度的目的，需要改写Prompt，尝试得到如下的Prompt，效果基本相似。

"""

from pydantic import BaseModel, Field
from typing import Optional, List, Annotated

class SubQuestion(BaseModel):
    id: int = Field(description="子问题的编号id，以1开始逐渐递增")
    text: str = Field(description="子问题")
    dependencies: List[int] = Field(description="依赖子问题的编号id列表")
    focus: str = Field(description="子问题关注的核心问题")

class QueryDecomposition(BaseModel):
    main_question: str = Field(description="用户输入的复杂问题")
    sub_questions: List[SubQuestion] = Field(description="复杂问题分解的子问题列表")



def create_query_plan(query):

    system_prompt = "You are a helpful assistant."

    system_prompt = """ 你是一个用户查询拆分的专家，擅长针对KBQA和搜索领域的查询进行分解。
    ### 任务说明：
    将用户的复杂查询分解为多个独立且易回答的子问题，输出 JSON 格式，保证子问题：
    1. 覆盖查询核心意图。
    2. 独立明确，可直接回答或检索。
    3. 标明逻辑依赖关系（如有）。
    4. 确保子问题与原始
    
    ### 仅输出json格式的内容，不要输出其它无关内容，输出json格式：
    ```
    {
      "main_question": "<用户复杂问题>",
      "sub_questions": [
        {
          "id": 1,
          "text": "<子问题1>",
          "dependencies": [],
          "focus": "<子问题关注的核心内容>"
        },
        {
          "id": 2,
          "text": "<子问题2>",
          "dependencies": [1],
          "focus": "<子问题关注的核心内容>"
        }
      ]
    }

    """

    
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content":  query},
    ]
    chat_response = client.chat.completions.create(model="Qwen2.5-7B-Instruct",messages=messages, 
                                                   temperature=0.001,
                                                   # top_p=0.8,
                                                   max_tokens=1024,
                                                   extra_body={
                                                               "repetition_penalty": 1.05,
                                                       # "guide_json": QueryDecomposition.model_json_schema(),
                                                       # "guided_decoding_backend": "outlines"
                                                   }
                                                  )

    
    
    return chat_response.choices[0].message.content



In [7]:
import json

# query = "功率最大的发电机是什么时候购买的"
query = "姚明和奥尼尔谁的身高高"
# query = "耗电量最大的设备是是什么时候在什么地方购买的"
# query = "张三的父亲的母亲多大了？"
# query = "谁创建了阿里"
# query = "治疗费用最高的患者的主治医生是男的还是女的？"
# query = "购买费用最高的设备是什么"


query_list = ["功率最大的发电机是什么时候购买的", "姚明和奥尼尔谁的身高高",
             "耗电量最大的设备是是什么时候在什么地方购买的", "张三的父亲的母亲多大了？",
             "谁创建了阿里", "治疗费用最高的患者的主治医生是男的还是女的？",
             "购买费用最高的设备是什么", "第五届和第六届奥运会的举办国家是谁",
             "高通和阿里的年度净利润谁高", "中国的GDP今年比去年高百分之几？"]

def parse_response(response: str):
    response = create_query_plan(query)
    # print("---------------LLM的原始输出--------------------------------")
    # print(response)
    json_string = response.strip('```json').strip('```')
    # print("---------------LLM的json解码输出--------------------------------")
    response_dict = json.loads(json_string)
    # print(response_dict)
    
    return response_dict
    
from pprint import pprint

for query in query_list[:]:
    
    response_dict = parse_response(query)
    print("----------------------------------------------------------")
    print("原始query: ", query)
    pprint("解析后的答案：")
    pprint(response_dict)
    print("----------------------------------------------------------")
    
    

----------------------------------------------------------
原始query:  功率最大的发电机是什么时候购买的
'解析后的答案：'
{'main_question': '功率最大的发电机是什么时候购买的',
 'sub_questions': [{'dependencies': [],
                    'focus': '最大功率发电机的基本信息',
                    'id': 1,
                    'text': '目前已知的最大功率发电机的型号和制造商是什么？'},
                   {'dependencies': [1],
                    'focus': '最大功率发电机的购买或使用时间',
                    'id': 2,
                    'text': '该最大功率发电机的购买时间或首次投入使用的时间是什么时候？'}]}
----------------------------------------------------------
----------------------------------------------------------
原始query:  姚明和奥尼尔谁的身高高
'解析后的答案：'
{'main_question': '姚明和奥尼尔谁的身高高',
 'sub_questions': [{'dependencies': [],
                    'focus': '姚明的身高',
                    'id': 1,
                    'text': '姚明的身高是多少？'},
                   {'dependencies': [],
                    'focus': '奥尼尔的身高',
                    'id': 2,
                    'text': '奥尼尔的身高是多少？'},
                   {'dependencie